In [3]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize 
from string import punctuation
from nltk.corpus import stopwords
import nltk
import ssl
from time import time
from textblob import TextBlob
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.linear_model import Ridge


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score,classification_report
from datetime import datetime

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nikhilagrawal/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nikhilagrawal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### Read the dataset

In [4]:
reviews_filename = 'Finalized_Reviews.csv'
users_filename = 'Finalized_users.csv'
restaurants_filename = 'Finalized_Restaurants.csv'

def prepareData(reviews_filename,users_filename,restaurants_filename):
    reviewData = pd.read_csv(reviews_filename)
    userData = pd.read_csv(users_filename)
    restaurantData = pd.read_csv(restaurants_filename)
    restaurantData = restaurantData.loc[restaurantData['business_id'].isin(reviewData['business_id'])]
    userData = userData.loc[userData['user_id'].isin(reviewData['user_id'])]
   
    return reviewData,userData,restaurantData

In [5]:
reviewData,userData,restaurantData = prepareData(reviews_filename,users_filename,restaurants_filename)

In [6]:
len(reviewData)

10000

In [7]:
len(userData)

7530

In [8]:
len(restaurantData)

4989

In [9]:
sid = SentimentIntensityAnalyzer()

In [11]:
def getSentimentScore(reviewData):
    sid = SentimentIntensityAnalyzer()
    pos = []
    neg = []
    for text in reviewData['text']:
        score = sid.polarity_scores(text)
        pos.append(score['pos'])
        neg.append(score['neg'])
    reviewData['PostiveScore'] = pos
    reviewData['NegativeScore'] = neg
    return reviewData

In [12]:
reviewData = getSentimentScore(reviewData)

In [18]:
fea = fea[ ]
reviewData.head()

,Unnamed: 0,user_id,review_id,text,business_id,stars,date,PostiveScore,NegativeScore
0,186584,GnKJvN6gCvFWTTvypohY4w,ATvoHIZNj1PQr4BW9BlSpQ,Absolutely love this place! \nGo on a Friday n...,ZONtqsnEl47Zn2NunJYHIg,5,2014-02-04,0.319,0.027
1,158482,rL3gE0RcqtnbMCyMZjNCzw,o8Wac77vmm_rpaJ5jpgOyg,Surprised this place has so little reviews and...,VxnZy3cU-ItbYiDK5XDX8g,4,2015-02-26,0.176,0.025
2,206140,dlHjGteH_47FBdgqyeQEIA,Ihq35D9pR7SOqxlg7bZF7w,"I have came here 3 times over the past year, b...",hDy-uY7Vy_TZdGBzw59lhA,1,2018-02-20,0.081,0.130
3,59415,SsOiVav4V5_NjTl21Lj92w,8-2bJeJ_26S_xn4NFjGmlw,My friend jus came back into town and the firs...,NoT7u2QNEvsVbH9MIzAzlw,4,2017-08-04,0.246,0.000
4,116143,8EbU9psAqfyqOdXSvyBizQ,TOe0LBRDZIdPH4SUyvKryA,I am shocked that this place has the number of...,-chGaXHZsoLKLHPJMMsdmw,1,2011-10-14,0.083,0.073


In [19]:
feat = ['text','PostiveScore','NegativeScore']
demo = reviewData[feat]

In [21]:
demo.head(5)

,text,PostiveScore,NegativeScore
0,Absolutely love this place! \nGo on a Friday n...,0.319,0.027
1,Surprised this place has so little reviews and...,0.176,0.025
2,"I have came here 3 times over the past year, b...",0.081,0.130
3,My friend jus came back into town and the firs...,0.246,0.000
4,I am shocked that this place has the number of...,0.083,0.073


In [22]:
def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]

def remove_stop_words(docs):
    en_stops = stopwords.words('english')
    en_stops.extend(['should','they','this','came','would','could'])
    new_docs = []
    for doc in docs:
        new_word = []  
        for word in doc:
            if word not in en_stops:
                new_word.append(word)
        new_docs.append(new_word)
            
    return new_docs

def filterInput(documents):
    new_docs = []
    for doc in documents:
        new_word = []
        for word in doc:
            new_word.append(word.lower())
            for char in word:
                if(not char.isalpha()):
                    new_word.remove(word.lower())
                    break
        new_docs.append(new_word)
    
    return new_docs

import re
def remove_punctuation(docs):
    new_docs = []
    for doc in docs:
        new_words = []  
        for word in doc:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        new_docs.append(new_words)
            
    return new_docs

In [37]:
def preprocessText(reviewData):
    docs =  list(reviewData['text'])
    docs_tokens = [word_tokenize(doc) for doc in docs] 
    docs_fil = filterLen(docs_tokens,4)
    docs_filt = filterInput(docs_fil)
    docs_wo_punctuation = remove_punctuation(docs_filt)
    preprocessed_docs = remove_stop_words(docs_wo_punctuation)
    return preprocessed_docs

In [38]:
pre = preprocessText(reviewData)

In [39]:
def createDictForLDA(preprocessed_docs):
    #Create Corpus
    texts = preprocessed_docs
    # Create Dictionary
    id2word = corpora.Dictionary(preprocessed_docs)
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    return id2word,corpus

In [40]:
id2word,corpus = createDictForLDA(pre)

In [45]:
# Build LDA model
start = time()
lda_model = gensim.models.LdaMulticore(workers=3,corpus=corpus,id2word=id2word, num_topics=10, random_state=100,passes=10)
end = time()
# Print the Keyword in the 15 topics
pprint(lda_model.show_topics())
print((end - start)/60)

[(0,
  '0.018*"burger" + 0.016*"good" + 0.014*"fries" + 0.014*"food" + '
  '0.010*"salad" + 0.009*"place" + 0.007*"really" + 0.007*"lunch" + '
  '0.007*"service" + 0.007*"great"'),
 (1,
  '0.036*"food" + 0.018*"service" + 0.013*"place" + 0.011*"good" + '
  '0.009*"restaurant" + 0.008*"great" + 0.007*"staff" + 0.007*"like" + '
  '0.007*"always" + 0.006*"back"'),
 (2,
  '0.015*"food" + 0.015*"place" + 0.011*"order" + 0.009*"time" + '
  '0.008*"service" + 0.008*"good" + 0.008*"really" + 0.006*"minutes" + '
  '0.006*"said" + 0.006*"ordered"'),
 (3,
  '0.030*"pizza" + 0.012*"good" + 0.009*"great" + 0.008*"like" + 0.008*"pasta" '
  '+ 0.008*"cheese" + 0.007*"fresh" + 0.007*"place" + 0.006*"sauce" + '
  '0.006*"really"'),
 (4,
  '0.011*"food" + 0.009*"time" + 0.009*"place" + 0.009*"good" + 0.009*"like" + '
  '0.008*"back" + 0.007*"service" + 0.007*"really" + 0.007*"table" + '
  '0.006*"asked"'),
 (5,
  '0.035*"sushi" + 0.014*"place" + 0.011*"roll" + 0.010*"sashimi" + '
  '0.010*"good" + 0.010

In [46]:
topic_pred = []
for i in range(0, len(reviewData.text)):
    temp = lda_model[corpus[i]]
    result = sorted(temp,key=lambda x:(-x[1],x[0]))     
    topic_pred.append(result[0][0])
reviewData['PredictedTopic'] = topic_pred

In [47]:
reviewData.head(10)

,Unnamed: 0,user_id,review_id,text,business_id,stars,date,PostiveScore,NegativeScore,PredictedTopic,PredictedRating
0,186584,GnKJvN6gCvFWTTvypohY4w,ATvoHIZNj1PQr4BW9BlSpQ,Absolutely love this place! \nGo on a Friday n...,ZONtqsnEl47Zn2NunJYHIg,5,2014-02-04,0.319,0.027,7,4.277979
1,158482,rL3gE0RcqtnbMCyMZjNCzw,o8Wac77vmm_rpaJ5jpgOyg,Surprised this place has so little reviews and...,VxnZy3cU-ItbYiDK5XDX8g,4,2015-02-26,0.176,0.025,1,3.641468
2,206140,dlHjGteH_47FBdgqyeQEIA,Ihq35D9pR7SOqxlg7bZF7w,"I have came here 3 times over the past year, b...",hDy-uY7Vy_TZdGBzw59lhA,1,2018-02-20,0.081,0.130,4,2.347512
3,59415,SsOiVav4V5_NjTl21Lj92w,8-2bJeJ_26S_xn4NFjGmlw,My friend jus came back into town and the firs...,NoT7u2QNEvsVbH9MIzAzlw,4,2017-08-04,0.246,0.000,4,4.165869
4,116143,8EbU9psAqfyqOdXSvyBizQ,TOe0LBRDZIdPH4SUyvKryA,I am shocked that this place has the number of...,-chGaXHZsoLKLHPJMMsdmw,1,2011-10-14,0.083,0.073,4,2.823182
5,12143,Axh-cOUoA715qgaem50stA,OHxLx-I4DuLMdLKesSRXdw,Miso soup was amazing!\n\nSpicy edamame was no...,0a2O150ytxrDjDzXNfRWkA,4,2017-02-02,0.394,0.117,5,3.883369
6,166836,gxRw9c2lvijXBNhKU5i7_g,37odqQZcQmzev643NhjNPw,Best meal I've had in a loonngggggggg time and...,hkZh4tAa2oJ34i8hxej6Zg,5,2013-11-16,0.454,0.000,9,5.116231
7,203118,2rPZkEQmsc15pQBrhZIikQ,b6ldX0PNq5rh3G_-BTuivg,"Yum yum yummy! I've only been here once, but i...",nRZjrWIzqKc26KjpBG0fHg,4,2016-07-14,0.264,0.000,7,4.248078
8,167832,TjvVhw2s9fesdFbBg_DiiQ,hzuIYui-MIHJ423aDHelRQ,This used to be a decent place to get Vietnam...,DF_w5Hwgi4bLgZH8_6F3cA,1,2016-09-05,0.035,0.061,2,2.702289
9,33227,MfwaL4gEaVpMs2IXydcuyQ,_WwSMhZFSqtaColDgRTVtA,Best taco in Toronto. Always a line. The fre...,iGEvDk6hsizigmXhDKs2Vg,4,2016-03-12,0.431,0.000,9,5.011195


In [48]:
from collections import defaultdict
topic_proba_map = defaultdict(list)
for i in range(0,10):
    lisd = []
    for val,pro in lda_model.get_topic_terms(i):
        lisd.append(pro)
    topic_proba_map[i]=lisd

In [49]:
features = pd.DataFrame()
start = time()

features['PositiveScore'] = reviewData['PostiveScore']
features['NegativeScore'] = reviewData['NegativeScore']


TP1 = []
TP2 = []
TP3 = []
TP4 = []
TP5 = []
TP6 = []
TP7 = []
TP8 = []
TP9 = []
TP10 = []
TN1 = []
TN2 = []
TN3 = []
TN4 = []
TN5 = []
TN6 = []
TN7 = []
TN8 = []
TN9 = []
TN10 = []
for j,row in reviewData.iterrows():
    ps = row['PostiveScore']
    ns = row['NegativeScore']
    temp = lda_model.get_topic_terms(row['PredictedTopic'])
    TP1.append(temp[0][1] * ps)
    TP2.append(temp[1][1] * ps)
    TP3.append(temp[2][1] * ps)
    TP4.append(temp[3][1] * ps)
    TP5.append(temp[4][1] * ps)
    TP6.append(temp[5][1] * ps)
    TP7.append(temp[6][1] * ps)
    TP8.append(temp[7][1] * ps)
    TP9.append(temp[8][1] * ps)
    TP10.append(temp[9][1] * ps)
    TN1.append(temp[0][1] * ns)
    TN2.append(temp[1][1] * ns)
    TN3.append(temp[2][1] * ns)
    TN4.append(temp[3][1] * ns)
    TN5.append(temp[4][1] * ns)
    TN6.append(temp[5][1] * ns)
    TN7.append(temp[6][1] * ns)
    TN8.append(temp[7][1] * ns)
    TN9.append(temp[8][1] * ns)
    TN10.append(temp[9][1] * ns)
features['TP1'] = TP1
features['TP2'] = TP2
features['TP3'] = TP3
features['TP4'] = TP4
features['TP5'] = TP5
features['TP6'] = TP6
features['TP7'] = TP7
features['TP8'] = TP8
features['TP9'] = TP9
features['TP10'] = TP10
features['TN1'] = TN1
features['TN2'] = TN2
features['TN3'] = TN3
features['TN4'] = TN4
features['TN5'] = TN5
features['TN6'] = TN6
features['TN7'] = TN7
features['TN8'] = TN8
features['TN9'] = TN9
features['TN10'] = TN10
end = time()
print((end - start)/60)

0.22312314907709757


In [50]:
features.head(10)

,PositiveScore,NegativeScore,TP1,TP2,TP3,TP4,TP5,TP6,TP7,TP8,...,TN1,TN2,TN3,TN4,TN5,TN6,TN7,TN8,TN9,TN10
0,0.319,0.027,0.005972,0.005325,0.004287,0.003963,0.003278,0.002304,0.002269,0.002258,...,0.000505,0.000451,0.000363,0.000335,0.000277,0.000195,0.000192,0.000191,0.000184,0.000167
1,0.176,0.025,0.006385,0.003194,0.002236,0.001877,0.001585,0.001451,0.001317,0.001202,...,0.000907,0.000454,0.000318,0.000267,0.000225,0.000206,0.000187,0.000171,0.000165,0.000145
2,0.081,0.130,0.000905,0.000767,0.000750,0.000735,0.000691,0.000662,0.000588,0.000558,...,0.001452,0.001231,0.001204,0.001179,0.001109,0.001063,0.000944,0.000895,0.000872,0.000844
3,0.246,0.000,0.002747,0.002329,0.002278,0.002232,0.002099,0.002011,0.001786,0.001694,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.083,0.073,0.000927,0.000786,0.000769,0.000753,0.000708,0.000679,0.000603,0.000571,...,0.000815,0.000691,0.000676,0.000662,0.000623,0.000597,0.000530,0.000503,0.000490,0.000474
5,0.394,0.117,0.013905,0.005647,0.004510,0.003981,0.003967,0.003917,0.003738,0.003678,...,0.004129,0.001677,0.001339,0.001182,0.001178,0.001163,0.001110,0.001092,0.001035,0.000878
6,0.454,0.000,0.007237,0.005539,0.005467,0.005025,0.004698,0.003786,0.003474,0.003378,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.264,0.000,0.004942,0.004407,0.003548,0.003280,0.002713,0.001906,0.001877,0.001868,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.035,0.061,0.000529,0.000510,0.000391,0.000330,0.000274,0.000274,0.000273,0.000213,...,0.000922,0.000889,0.000681,0.000575,0.000478,0.000477,0.000476,0.000370,0.000353,0.000348
9,0.431,0.000,0.006871,0.005258,0.005190,0.004771,0.004460,0.003594,0.003298,0.003207,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [51]:
X = features
y = reviewData['stars']
X_train , X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=42)
clf = Ridge(alpha=1.0)
clf.fit(X_train, y_train) 
pred_rating = clf.predict(X_test)
#reviewData['PredictedRating'] = pred_rating
print(metrics.mean_absolute_error(y_test,pred_rating))
print(metrics.mean_squared_error(y_test,pred_rating))

0.8065726691082155
0.9929222108864284


In [34]:
X = features
y = reviewData['stars']
#X_train , X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=42)
clf = Ridge(alpha=1.0)
clf.fit(X, y) 
pred_rating = clf.predict(X)
reviewData['PredictedRating'] = pred_rating
# print(metrics.mean_absolute_error(y_test,pred_rating))
# print(metrics.mean_squared_error(y_test,pred_rating))

In [36]:
reviewData.head(5)

,Unnamed: 0,user_id,review_id,text,business_id,stars,date,PostiveScore,NegativeScore,PredictedTopic,PredictedRating
0,186584,GnKJvN6gCvFWTTvypohY4w,ATvoHIZNj1PQr4BW9BlSpQ,Absolutely love this place! \nGo on a Friday n...,ZONtqsnEl47Zn2NunJYHIg,5,2014-02-04,0.319,0.027,5,4.277979
1,158482,rL3gE0RcqtnbMCyMZjNCzw,o8Wac77vmm_rpaJ5jpgOyg,Surprised this place has so little reviews and...,VxnZy3cU-ItbYiDK5XDX8g,4,2015-02-26,0.176,0.025,8,3.641468
2,206140,dlHjGteH_47FBdgqyeQEIA,Ihq35D9pR7SOqxlg7bZF7w,"I have came here 3 times over the past year, b...",hDy-uY7Vy_TZdGBzw59lhA,1,2018-02-20,0.081,0.130,2,2.347512
3,59415,SsOiVav4V5_NjTl21Lj92w,8-2bJeJ_26S_xn4NFjGmlw,My friend jus came back into town and the firs...,NoT7u2QNEvsVbH9MIzAzlw,4,2017-08-04,0.246,0.000,5,4.165869
4,116143,8EbU9psAqfyqOdXSvyBizQ,TOe0LBRDZIdPH4SUyvKryA,I am shocked that this place has the number of...,-chGaXHZsoLKLHPJMMsdmw,1,2011-10-14,0.083,0.073,4,2.823182
